# Automatic Differentiation with AutoGrad

When training neural networks, the most frequently used algorithm is **back propagation**. In this algorithm, parameters (model weights) are adjusted according to the **gradient** of the loss function with respect to the given parameter.

To compute those gradients, PyTorch has a built-in mechanism called **AutoGrad**. It supports automatic computation of gradient for any computational graph.

### Tensors, Functions and Computational graph

The main function of PyTorch is to perform fast computations on [tensors](tensors.ipynb). Consider the following piece of code:

In [29]:
import torch
x = torch.ones(2,2,requires_grad=True)
y = x+2
print(x,y,sep='\n')

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In this code, we first define a 2x2 tensor `x`, and then compute tensor `y` based on the value of `x`. This defines the following **computational graph**:

<img height="40" src="images/autograd-graph-1.png"/>

When defining original tensor, we have set `requires_grad` property to `True`, which means that we want to be able to track gradients for all expressions including this tensor. Thus, when we define tensor `y` that depends on `x`, PyTorch engine does not only compute the value of `y`, but it also keeps the track of the function used to compute it. This function is available as `grad_fn` property of the tensor.

> **Note:** You can set the value of `requires_grad` when creating a tensor, or later by using `x.requires_grad_(True)` method.

A function that we apply to tensors to construct computational graph is in fact an object of class `Function`. This object knows how to compute the function in the *forward* direction, and also how to compute it's derivative during the *backward propagation* step. A reference to the backward propagation function is stored in `grad_fn` property of a tensor. You can find more information of `Function` [in documentation](https://pytorch.org/docs/stable/autograd.html#function).

Now let's build more complex computational graph like this:

<img height="40" src="images/autograd-graph-2.png"/>

In [30]:
z = y*y*3
out = z.mean()
print(z,out,sep='\n')

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


## Computing Gradients

We have defined `out` as a function of `x`. Now suppose we want to compute the gradient $\partial out\over\partial x$. To do that, we first need to call `out.backward()`, which will compute all the gradients back from the `out` node of our graph using backward propagation. Then, we can access `x.grad` property to retrieve the gradient:

In [31]:
out.backward()
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


Let's try to understand where this value comes from. 

If we have a vector function $\vec{y}=f(\vec{x})$, where $\vec{x}=\langle x_1,\dots,x_n\rangle$ and $\vec{y}=\langle y_1,\dots,y_m\rangle$, then a gradient of $\vec{y}$ with respect to $\vec{x}$ is given by so-called **Jacobian matrix**:

$$
\begin{align}J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\end{align}
$$

If $y(\vec{x})$ is a scalar function, than the gradient becomes $\left(\frac{\partial y}{x_1} \dots \frac{\partial y}{x_n}\right)$, and if vector $\vec{x}$ is a tensor of higher dimensionality, gradient also becomes a tensor of the same size.

In our case, we have $2\times2$ tensor:
$$
x = \left(\begin{array}{cc}x_{11} & x_{12} \\ x_{21} & x_{22}\end{array}\right), \mbox{ and  }
\frac{\partial out}{\partial x} = \left(\begin{array}{cc}\frac{\partial out}{\partial x_{11}} & \frac{\partial out}{\partial x_{12}} \\ \frac{\partial out}{\partial x_{21}} & \frac{\partial out}{\partial x_{22}}\end{array}\right)
$$

Let us now compute the derivative. Our output is
$$
out = \frac{1}{4}\sum_{ij} z_{ij} = \frac{1}{4}\sum_{ij}3(x_{ij}+2)^2
$$
Therefore,
$$
\frac{\partial out}{\partial x_{ij}} = \frac{3}{2}(x_{ij}+2)
$$

PyTorch computes the derivative value at a specific point $x=\left(\begin{array}{cc}1&1\\ 1&1\end{array}\right)$, so
$$
\left.{\partial out\over\partial x_{ij}}\right|_{x_{ij}=1} = \sum_t{3\over2}(1+2) = {9\over2} = 4.5
$$

**Note:** We can only obtain the `grad` properties for the leaf nodes of the computational graph. This is because in real life we only need those gradients that are related to neural network parameters, and not for the intermediate values inside the computational graph. And those parameters - weights - are defined as free tensors that represent leaves of the computational graph.

In [54]:
print(z.grad)

None


## Computing Jacobian Products

In the previous case, we had a scalar value `out` depending on the input tensor `x`, and we computed partial derivatives of this value with respect to individual tensor coordinates. In more general case, when we have a tensor function from another tensor, a Jacobian matrix becomes more complex. Instead of computing the Jacobian itself, PyTorch allows you to compute **Jacobian Product** $v^T\cdot J$ for a given input vector $v=(v_1 \dots v_m)$. 

This often makes sense, because if we have a vector function $\vec{y}=f(\vec{x})$, and then compute some scalar function $l(\vec{y})$, then by passing $v=(\frac{\partial l}{y_1} \dots \frac{\partial l}{y_m})^T$ to the Jacobian product we can compute the gradient of $l$ with respect to $\vec{x}$:
$$
\begin{align}J^{T}\cdot v=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\left(\begin{array}{c}
   \frac{\partial l}{\partial y_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial y_{m}}
   \end{array}\right)=\left(\begin{array}{c}
   \frac{\partial l}{\partial x_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial x_{n}}
   \end{array}\right)\end{align}
$$

In our case, suppose we want to compute the Jacobian product for `z`. In this case, we need to pass the tensor $v$ as an argument to the `z.backward` function:

In [51]:
z = (x+2).pow(2)*3
z.backward(torch.ones_like(x),retain_graph=True)
print(x.grad)

tensor([[18., 18.],
        [18., 18.]])


**Note:** Here we needed to re-define the computational graph from scratch (and did it in slightly shorter way). This is because normally computational graphs supports only one `backward` step, unless we explicitly specify `retain_graph=True` on calls to `backward`. 

Notice that if we call `backward` once more, the value of the gradients would be different: 

In [52]:
z.backward(torch.ones(2,2),retain_graph=True)
print(x.grad)

tensor([[36., 36.],
        [36., 36.]])


This happens because when doing `backward` propagation, PyTorch **accumulates the gradients**, i.e. the value of computed gradients is added to the `grad` property of all leaf nodes of computational graph. If you want to compute the proper gradients, you need to zero out the `grad` property before. In real-life training an *optimizer* helps us do this. 

In [53]:
x.grad.zero_()
z.backward(torch.ones_like(x))
print(x.grad)

tensor([[18., 18.],
        [18., 18.]])


**Note:** Previously we were calling `backward()` function without parameters. This is essentially equivalent to calling `backward(torch.tensor(1.0))`, which is a useful way to compute the gradients in case of a scalar-valued function, such as loss during neural network training.

## Disabling Gradient Tracking

By default, all tensors with `requires_grad=True` are tracking their computational history and support gradient computation. However, there are some cases when we do not need to do that, for example, when we have trained the model and just want to apply it to some input data, i.e. we only want to do *forward* computations through the network. We can stop tracking computations by surrounding our computation code with `with torch.no_grad()` block:

In [56]:
z = x*x
print(z.requires_grad)

with torch.no_grad():
    z = x*x
print(z.requires_grad)

True
False


Another way to achieve the same result is to use the `detach()` method on the tensor:

In [57]:
z = x*x
z_det = z.detach()
print(z_det.requires_grad)

False


All forward-pass computations on tensors that do not track gradients would be more efficient.

## Example of Gradient Descent

Let's use the AutoGrad functionality to minimize a simple function of two variables $f(x_1,x_2)=(x_1-3)^2+(x_2+2)^2$. We will use the `x` tensor to represent the coordinates of a point. To do the gradient descent, we start with some initial value $x^{(0)}=(0 0)$, and compute each consecutive step using:
$$
x^{(n+1)} = x^{(n)} - \eta\nabla f
$$
Here $\eta$ is so-called **learning rate** (we will call it `lr` in our code), and $\nabla f = (\frac{\partial f}{\partial x_1},\frac{\partial f}{\partial x_2})$ is the gradient of $f$.

We will start by defining the initial value of `x` and the function `f`:

In [100]:
x = torch.zeros(2,requires_grad=True)
f = lambda x : (x-torch.tensor([3,-2])).pow(2).sum()
lr = 0.1

For the gradient descent, let's do 15 iterations. On each iteration, we will update the coordinate tensor `x` and print its coordinates to make sure that we are approaching the minimum:

In [101]:
for i in range(15):
    y = f(x)
    y.backward()
    gr = x.grad
    x.data.add_(-lr*gr)
    x.grad.zero_()
    print("Step {}: x[0]={}, x[1]={}".format(i,x[0],x[1]))

Step 0: x[0]=0.6000000238418579, x[1]=-0.4000000059604645
Step 1: x[0]=1.0800000429153442, x[1]=-0.7200000286102295
Step 2: x[0]=1.4639999866485596, x[1]=-0.9760000705718994
Step 3: x[0]=1.7711999416351318, x[1]=-1.1808000802993774
Step 4: x[0]=2.0169599056243896, x[1]=-1.3446400165557861
Step 5: x[0]=2.2135679721832275, x[1]=-1.4757120609283447
Step 6: x[0]=2.370854377746582, x[1]=-1.5805696249008179
Step 7: x[0]=2.4966835975646973, x[1]=-1.6644556522369385
Step 8: x[0]=2.597346782684326, x[1]=-1.7315645217895508
Step 9: x[0]=2.677877426147461, x[1]=-1.7852516174316406
Step 10: x[0]=2.7423019409179688, x[1]=-1.8282012939453125
Step 11: x[0]=2.793841600418091, x[1]=-1.8625609874725342
Step 12: x[0]=2.835073232650757, x[1]=-1.8900487422943115
Step 13: x[0]=2.868058681488037, x[1]=-1.912039041519165
Step 14: x[0]=2.894446849822998, x[1]=-1.929631233215332


As you can see, we have obtained the values close to the optimal point $(3,-2)$. Training a neural network is in fact a very similar process, we will need to do a number of iterations to minimize the value of **loss function**.